In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = {}
y = {}
X["train"] = np.load("X_train.npy",allow_pickle=True )
y["train"] = np.load("y_train.npy")
X["dev"] = np.load("X_dev.npy")
y["dev"] = np.load("y_dev.npy")
X_text_train = np.load("X_text_train.npy")
X_text_dev = np.load("X_text_dev.npy")
X["train"].shape, X["dev"].shape

((11678, 300), (1298, 300))

In [2]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.9653312788906009

In [3]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((11678, 301), (1298, 301))

In [4]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 1 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [5]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1298, 11678)

In [6]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    print("prediction", prediction)

    return X_k, y_k, prediction

In [7]:
ls

CSV_essay_LR_alg1.csv    X_test.npy               essay_dev.tsv
CSV_essay_LR_alg1.ipynb  X_text_dev.npy           essay_train.tsv
Errors_k.ipynb           X_text_train.npy         new_pred.npy
Smallest_k.ipynb         X_train.npy              npy2tsv.ipynb
Smallest_k.py            appro_ks.npy             old_prediction.npy
Untitled.ipynb           asap-aes/                output.tsv
Use_approK.ipynb         data/                    y_dev.npy
X_dev.npy                data_loader.ipynb        y_train.npy


In [8]:
app_k = np.array(np.load("appro_ks.npy", allow_pickle=True).squeeze())
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if (not app_k[i] > 0):
        app_k_nonon.append(X["train"].shape[0])
    else:
        app_k_nonon.append(app_k[i])

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
53.0
nan
550.0
nan
nan
nan
nan
462.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
22.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
4336.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
511.0
1864.0
nan
nan
35.0
nan
471.0
nan
nan
nan
nan
nan
nan
115.0
nan
nan
3.0
nan
nan
238.0
nan
nan
nan
nan
nan
nan
251.0
nan
nan
201.0
nan
nan
420.0
nan
nan
129.0
nan
nan
nan
nan
nan
nan
nan
nan
32.0
230.0
170.0
nan
nan
191.0
nan
13.0
221.0
nan
nan
nan
518.0
nan
nan
864.0
894.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
770.0
nan
nan
nan
20.0
nan
747.0
nan
nan
nan
nan
nan
nan
nan
561.0
76.0
nan
nan
nan
nan
nan
nan
889.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
106.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
322.0
nan
nan
nan
nan
44.0
nan
nan
nan
nan
nan
35.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

In [9]:
sort_dev_k = np.array(app_k_nonon).argsort()
sort_dev_k

array([1161, 1257,  467, ...,  456,  463, 1297])

In [10]:
app_k_nonon[sort_dev_k[0]]

1.0

In [19]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [20]:
new_predictions = np.load("new_pred.npy", allow_pickle=True)

In [22]:
data = []
for test_movie_id in sort_dev_k:
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if not (k >= 0):
        test_point.append(None)
        data.append(test_point)
        continue
    k = int(k)
    test_point.append(np.round(new_predictions[test_movie_id].item()) != np.round(pred[test_movie_id].item()))
        
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)

k 1.0
k 2.0
k 2.0
k 3.0
k 3.0
k 5.0
k 5.0
k 6.0
k 7.0
k 7.0
k 10.0
k 10.0
k 12.0
k 12.0
k 13.0
k 14.0
k 14.0
k 14.0
k 15.0
k 17.0
k 19.0
k 20.0
k 21.0
k 21.0
k 21.0
k 22.0
k 25.0
k 25.0
k 26.0
k 27.0
k 27.0
k 28.0
k 30.0
k 32.0
k 32.0
k 33.0
k 35.0
k 35.0
k 36.0
k 43.0
k 43.0
k 44.0
k 46.0
k 47.0
k 48.0
k 49.0
k 50.0
k 53.0
k 55.0
k 61.0
k 69.0
k 69.0
k 71.0
k 72.0
k 76.0
k 76.0
k 80.0
k 83.0
k 88.0
k 88.0
k 92.0
k 96.0
k 98.0
k 101.0
k 105.0
k 106.0
k 106.0
k 108.0
k 115.0
k 116.0
k 117.0
k 120.0
k 122.0
k 125.0
k 129.0
k 129.0
k 130.0
k 132.0
k 136.0
k 140.0
k 147.0
k 157.0
k 159.0
k 170.0
k 176.0
k 181.0
k 187.0
k 191.0
k 193.0
k 200.0
k 201.0
k 221.0
k 224.0
k 230.0
k 232.0
k 234.0
k 238.0
k 242.0
k 251.0
k 254.0
k 270.0
k 282.0
k 301.0
k 303.0
k 319.0
k 322.0
k 333.0
k 338.0
k 345.0
k 396.0
k 402.0
k 420.0
k 424.0
k 434.0
k 436.0
k 456.0
k 462.0
k 471.0
k 494.0
k 511.0
k 518.0
k 550.0
k 561.0
k 584.0
k 588.0
k 596.0
k 622.0
k 645.0
k 747.0
k 770.0
k 788.0
k 802.0
k 857.0
k 859.0
k

In [23]:
import csv

with open('CSV_essay_LR_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [18]:
ls

CSV_essay_LR_alg1.csv    X_test.npy               essay_dev.tsv
CSV_essay_LR_alg1.ipynb  X_text_dev.npy           essay_train.tsv
Errors_k.ipynb           X_text_train.npy         new_pred.npy
Smallest_k.ipynb         X_train.npy              npy2tsv.ipynb
Smallest_k.py            appro_ks.npy             old_prediction.npy
Untitled.ipynb           asap-aes/                output.tsv
Use_approK.ipynb         data/                    y_dev.npy
X_dev.npy                data_loader.ipynb        y_train.npy


In [44]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_SST/IF_lr_fix'